## <b><font color='darkblue'>Preface</font></b>
([article source](https://www.pinecone.io/learn/series/langchain/langchain-expression-language/)) <b><font size='3ptx'>The [LangChain Expression Language](https://python.langchain.com/v0.1/docs/expression_language/) (LCEL) is an abstraction of some interesting Python concepts into a format that enables a "minimalist" code layer for building chains of LangChain components.</font></b>

LCEL comes with strong support for:
1. Superfast development of chains.
2. Advanced features such as streaming, async, parallel execution, and more
3. Easy integration with [**LangSmith**](https://www.langchain.com/langsmith) and [**LangServe**](https://python.langchain.com/v0.2/docs/langserve/).

In this article, we'll learn what LCEL is, how it works, and the essentials of LCEL chains, pipes, and Runnables.

## <b><font color='darkblue'>LCEL Syntax</font></b>
We'll begin by installing all of the prerequisite libraries that we'll need for this walkthrough. Note, you can follow along via our [Jupyter notebook here](https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/11-langchain-expression-language.ipynb).

In [1]:
#!pip install -qU \
#    langchain \
#    anthropic \
#    cohere \
#    docarray

In [7]:
# pip install -U langchain-anthropic

In [63]:
#!pip install faiss-cpu

To understand LCEL syntax let's first build a simple chain using the traditional LangChain syntax. We will initialize a simple <b><font color='blue'>LLMChain</font></b> using Claude 2.1.

In [2]:
import os

from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
prompt = ChatPromptTemplate.from_template(
    "Give me small report about {topic}"
)
model = ChatAnthropic(
    model="claude-2.1",
    max_tokens_to_sample=512,
    anthropic_api_key=os.environ['ANTHROPIC_API_KEY'],
)  # swap Anthropic for OpenAI with `ChatOpenAI` and `openai_api_key`

Using this chain we can generate a small report about a particular topic, such as "Artificial Intelligence" by calling the <font color='blue'>chain.run</font> method on an [**LLMChain**](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html):

In [4]:
from langchain.chains import LLMChain
from langchain_community.chat_models.openai import ChatOpenAI

model_name = "gpt-3.5-turbo"

# Please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter
chain = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model_name),
    prompt=prompt,
    output_key="solutions"
)

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [5]:
# and run
out = chain.run(topic="Artificial Intelligence")

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [6]:
print(out)

Artificial intelligence (AI) is a rapidly advancing field of computer science that focuses on creating intelligent machines that can perform tasks that typically require human intelligence. These tasks include learning, reasoning, problem-solving, perception, and language understanding.

AI technologies are being used in a wide range of industries, including healthcare, finance, transportation, and entertainment. Some common applications of AI include virtual assistants like Siri and Alexa, self-driving cars, and recommendation systems used by companies like Amazon and Netflix.

There are different types of AI, including narrow AI, which is designed to perform specific tasks, and general AI, which aims to replicate human intelligence and perform a wide range of tasks. While narrow AI is already widely used, the development of general AI is still in its early stages.

Despite the many benefits of AI, there are also concerns about its potential impact on jobs, privacy, and ethics. As AI 

With [**LCEL**](https://python.langchain.com/v0.1/docs/expression_language/) we create our chain differently using pipe operators (`|`) rather than Chains:

In [21]:
model = ChatOpenAI(temperature=0, model=model_name)
output_parser = StrOutputParser()

lcel_chain = prompt | model | output_parser

In [20]:
# and run
out = lcel_chain.invoke({
    "topic": "Artificial Intelligence"})

In [21]:
print(out)

Artificial intelligence (AI) is a rapidly advancing field of computer science that focuses on creating intelligent machines that can perform tasks that typically require human intelligence. These tasks include speech recognition, decision-making, problem-solving, and learning.

AI systems are designed to analyze and interpret complex data, recognize patterns, and make decisions based on that information. They can be used in a wide range of applications, from self-driving cars and virtual assistants to medical diagnosis and financial forecasting.

One of the key components of AI is machine learning, which involves training algorithms to learn from data and improve their performance over time. This allows AI systems to adapt to new information and make more accurate predictions.

While AI has the potential to revolutionize many industries and improve efficiency and productivity, there are also concerns about the ethical implications of AI, including issues related to privacy, bias, and j

The syntax here is not typical for Python but uses nothing but native Python. <b>Our `|` operator simply takes output from the left and feeds it into the function on the right</b>.

## <b><font color='darkblue'>How the Pipe Operator Works</font></b>
<b><font size='3ptx'>To understand what is happening with LCEL and the pipe operator we create our own pipe-compatible functions.</font></b>

When the Python interpreter sees the `|` operator between two objects (<font color='brown'>like `a | b`</font>) it attempts to feed object `b` into the [\_\_or__](__or__) method of object `a`. That means these patterns are equivalent:
```python
# object approach
chain = a.__or__(b)
chain("some input")

# pipe approach
chain = a | b
chain("some input")
```

For example:

In [7]:
class ObjectWithOr:
    def __init__(self, value):
        self.value = value
        
    def __or__(self, obj):
        print(f'{self.value } Or {obj}')
        if isinstance(obj, ObjectWithOr):
            return ObjectWithOr(self.value + obj.value)
        else:
            return ObjectWithOr(self.value + obj)

    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        return str(self.value)

In [8]:
a = ObjectWithOr(1)
b = ObjectWithOr(2)

In [41]:
a | b

1 Or 2


3

In [9]:
a.__or__(b)

1 Or 2


3

With that in mind, we can build a <b><font color='blue'>Runnable</font></b> class that consumes a function and turns it into a function that can be chained with other functions using the pipe operator `|`. For example:

In [10]:
class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

Let's implement this to take the value `3`, add `5` (<font color='brown'>giving `8`</font>), and multiply by `2` — giving us `16`.

In [11]:
def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

# wrap the functions with Runnable
add_five = Runnable(add_five)
multiply_by_two = Runnable(multiply_by_two)

# run them using the object approach
chain = add_five.__or__(multiply_by_two)
chain(3)  # should return 16

16

Using `__or__` directly we get the correct answer, let's try using the pipe operator `|` to chain them together:

In [12]:
# chain the runnable functions together
chain = add_five | multiply_by_two

# invoke the chain
chain(3)  # we should return 16

16

With either method, we get the same response, and at its core, this is the pipe logic that [**LCEL**](https://python.langchain.com/v0.1/docs/expression_language/) uses when chaining together components. However, this is not all there is to [**LCEL**](https://python.langchain.com/v0.1/docs/expression_language/), there is more.

## <b><font color='darkblue'>LCEL Deep Dive</font></b>
Now that we understand what the LCEL syntax is doing under the hood, let's explore it within the context of LangChain and see a few of the additional methods that are provided to maximize flexibility when working with LCEL.

### <b><font color='darkgreen'>Runnables</font></b>
<b><font size='3ptx'>When working with LCEL we may find that we need to modify the flow of values, or the values themselves as they are passed between components</font></b> — for this, we can use runnables. Let's begin by initializing a couple of simple vector store components:
* [**API**:langchain_community.vectorstores.faiss.FAISS](https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.faiss.FAISS.html)

In [13]:
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [14]:
embedding_output =  embeddings_model.embed_query('test text')

In [15]:
embedding_output[:3]

[-0.00817158695845937, -0.0026620087657393135, -0.0003281987911357356]

In [16]:
def add_to_faiss_index(embeddings):
    vector = np.array(embeddings)
    index = faiss.IndexFlatL2(vector.shape[1])
    index.add(vector)
    return index


def get_embedding(text, embeddings_model):
    return embeddings_model.embed_query(text)

In [17]:
text_array1 = ["half the info will be here", "James' birthday is the 7th December"]
text_array2 = ["and half here", "James was born in 1994"]

In [18]:
faiss_1 = FAISS.from_texts(text_array1, embeddings_model)
faiss_2 = FAISS.from_texts(text_array2, embeddings_model)

We're creating two local vector stores here and breaking apart two essential pieces of information between the two vector stores. We'll see why soon, but for now we only need one of these. Let's try passing a question through a RAG pipeline using `faiss_1`

In [19]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = faiss_1.as_retriever()
retriever_b = faiss_2.as_retriever()

In [22]:
prompt_str = """Answer the question below using the context:

Context: {context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {"context": retriever_a, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | model | output_parser

We use two new objects here, [**RunnableParallel**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableParallel.html) and [**RunnablePassthrough**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html). The [**RunnableParallel**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableParallel.html) object allows us to define multiple values and operations, and run them all in parallel. Here we call `retriever_a` using the input to our chain (below), and then pass the results from `retriever_a` to the next component in the chain via the "`context`" parameter.

![fig1](images/fig_1.PNG)

The [**RunnablePassthrough**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html) object is used as a "`passthrough`" take takes any input to the current component (<font color='brown'>retrieval</font>) and allows us to provide it in the component output via the "`question`" key.

In [23]:
out = chain.invoke("When was James born and include the date in the answer?")

In [24]:
print(out)

James was born on the 7th December.


Using this information the chain is close to answering the question but it doesn't have enough information, it is missing the information (what year) that we have stored in `retriever_b`. Fortunately, we can have multiple parallel information streams with the [**RunnableParallel**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableParallel.html) object.

In [25]:
prompt_str = """Answer the question below using the context:

Context:
{context_a}
{context_b}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b,
        "question": RunnablePassthrough()
    }
)

chain = retrieval | prompt | model | output_parser

Here we're passing two sets of context to our prompt component via "`context_a`" and "`context_b`". Using this we get more information into our LLM (<font color='brown'>although oddly the LLM doesn't manage to put two-and-two together</font>).

In [26]:
out = chain.invoke("When was James born and include the date in the answer?")

In [27]:
print(out)

James was born on the 7th December 1994.


Using this approach we're able to have multiple parallel executions and build more complex chains pretty easily.

## <b><font color='darkblue'>Runnable Lambdas</font></b>
<b><font size='3ptx'>The [RunnableLambda](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html) is a LangChain abstraction that allows us to turn Python functions into pipe-compatible functions, similar to the [Runnable](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html) class we created near the beginning of this article.</font></b>

### <b><font color='darkgreen'>Example 1: Extended prompt</font></b>
Let's try it out with our earlier <font color='blue'>add_five</font> and <font color='blue'>multiply_by_two</font> functions.

In [28]:
from langchain_core.runnables import RunnableLambda

def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

# wrap the functions with RunnableLambda
add_five = RunnableLambda(add_five)
multiply_by_two = RunnableLambda(multiply_by_two)

As with our earlier [**Runnable**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html) abstraction, we can use <b><font color='orange'>|</font></b> operators to chain together our [**RunnableLambda**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html) abstractions.

In [29]:
chain = add_five | multiply_by_two

Unlike our [**Runnable**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html) abstraction, we cannot run the [**RunnableLambda**](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableLambda.html) chain by calling it directly, instead we must call <font color='blue'>chain.invoke</font>:

In [30]:
# (3 + 5) * 2 = 16
chain.invoke(3)

16

As before, we can see the same answer. <b>Naturally, we can feed custom functions into our chains using this approach. Let's try a short chain and see where we might want to insert a custom function</b>:

In [31]:
prompt_str = "Tell me an short fact about {topic}"
prompt = ChatPromptTemplate.from_template(prompt_str)

chain = prompt | model | output_parser

We can run this chain a couple of times to see what type of answers it returns:

In [32]:
chain.invoke({"topic": "Artificial Intelligence"})

'Artificial Intelligence is the simulation of human intelligence processes by machines, especially computer systems.'

Assume you want to extend the prompt to indicate the LLM to use a certain tone or lines or sentences to explain the topics. You could chain another prompt:

In [51]:
def add_context(data_dict):
    role = data_dict['role']
    num_lines = data_dict['num_lines']
    topic = data_dict['topic']
    context_str = f'Assume you are a {role}, please explain the topic in {num_lines} line(s).'
    return {'context': context_str, 'topic': topic}


context = RunnableLambda(add_context)

prompt_str = "{context}\nTell me an short fact about {topic}"
prompt = ChatPromptTemplate.from_template(prompt_str)

In [52]:
combined_prompt = context | prompt

In [53]:
combined_prompt.invoke({"role":"student", "num_lines":5, "topic":"Artificial Intelligence"})

ChatPromptValue(messages=[HumanMessage(content='Assume you are a student, please explain the topic in 5 line(s).\nTell me an short fact about Artificial Intelligence')])

Now let's see how it works in chain with LLM:

In [54]:
chain = context | prompt | model | output_parser

In [61]:
resp = chain.invoke({"role":"software engineer", "num_lines":3, "topic":"Artificial Intelligence"})

In [62]:
print(resp)

Artificial Intelligence is the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation.

Short fact: The term "artificial intelligence" was coined in 1956 by computer scientist John McCarthy.


### <b><font color='darkgreen'>Example 2: Audio to text</font></b>
When working with audio input, integrate an audio-to-text transcription step into your processing pipeline. This converts the audio file into its corresponding text, which can then be used as input for the LLM.

### <b><font color='darkgreen'>Conclusion</font></b>
<b>That covers the essentials you need to get started and build with the [LangChain Expression Language](https://python.langchain.com/v0.1/docs/expression_language/)</b> (LCEL). With it, we can put together chains very easily — and the current focus of the LangChain team is on further LCEL development and support.

The pros and cons of LCEL are varied. Those who love it tend to focus on the minimalist code style, LCEL's support for streaming, parallel operations, and async, and LCEL's nice integration with LangChain's focus on chaining components together.

<b>Some people are less fond of LCEL. These people typically point to LCEL being yet another abstraction on top of an already very abstract library, that the syntax is confusing, against the [Zen of Python](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fpeps.python.org%2Fpep-0020%2F)</b>, and requires too much effort to learn new (or uncommon) syntax.

Both viewpoints are entirely valid, LCEL is a very different approach — there are major pros and major cons. In either case, if you're willing to spend some time learning the syntax, it allows us to develop very quickly, and with that in mind, it's well worth learning.

## <b><font color='darkblue'>Supplement</font></b>
* [pinecone-io: learn/generation/langchain/handbook/11-langchain-expression-language.ipynb](https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/11-langchain-expression-language.ipynb)